In [1]:
import math, json
from tapipy.tapis import Tapis
from keras.models import Sequential, model_from_json
import pandas as pd


2024-03-20 01:45:08.911505: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [26]:
class DNNHelper:
    def __init__(self, signIn=False, credsFile=None):
        self.loginFile = credsFile
        self.TAPISclient = None
        if signIn:
            # CHANGE tghis to Token
            # Maybe push these t a YAML file? 
            data = json.load(open('TAPISCreds.json'))
            base_url=data['base_url']
            username=data['username']
            password=data['password']
            self.TAPISclient = Tapis(base_url= base_url, username=username, password=password)
            self.TAPISclient.get_tokens()
            # print("client.access_token", client.access_token)

            # Adding the Estimator APP
            self.app_def = json.load(open('EstimationJob.json'))
            self.TAPISclient.apps.putApp(**self.app_def, appId=self.app_def.id, appVersion="1.0.0")
        

    def getModelSummaryJSON(self, model):
       return model.to_json()
    

    def getSystemJSON(self, systemid, systemDesc, sysvalues):
        sysJSON = {
            "id": systemid,
            "host": sysvalues.host,
            "owner": sysvalues.owner,
            "effectiveUserId":  sysvalues.effectiveUserId, 
            "systemType": sysvalues.systemType
        }
        # Merge Code - Sams + additional Prediction Hardware Descritopion. 
        sysJSON["CPUType"] = "abc"
        sysJSON["GPUType"] = "xyz"
        return sysJSON
    
    #private
    def getSystems(self):
        # print("\t\t\t****************************************************")
        # print("\t\t\t\t\tList all systems")
        # print("\t\t\t****************************************************")
        my_systems = self.TAPISclient.systems.getSystems()
        # SPECS - > redfish -> control vocab.  Uniform Resuce descriptions SGCI ==> write the attributes. 
        # my_systems
        list_sys = []
        for idx, sys in enumerate(my_systems):
            sysDesc = self.TAPISclient.systems.getSystem(systemId=sys.id)
            sysJSON = self.getSystemJSON(sys.id, sysDesc, sys)
            
            list_sys.append(sysJSON)
        # df_sys = pd.DataFrame(list_sys, columns=["System ID", "Host", "Owner", "Effective User ID", "System Type"])
        return list_sys
    
    def fetchSystem4Config(self, app_requirement):
        executeOptions = self.getSystems()
        
        # Find the System Smallest System Which fits the Jobs - memery, cores and walltime
        return executeOptions[0]

    

    def createJobForEstimation(self):
        # Fixed for the estimation module
        requiremnets = { 
            "memoryMB": 44000,
            "nodeCount": 1,
            "coresPerNode": 10,
            "maxMinutes": 30 
            }
        systemID = self.fetchSystem4Config(requiremnets)

        jobargs = {
            "parameterSet": {
                "schedulerOptions": [
                        {"arg": "-A <PROJECRT_ID>" }
                    ],
            },
            
            # Can provide this is Job Parameteres Vs fileInputs
            "fileInputs": [
                {
                "name": "DNN_Modal_json",
                "description": "The JSON File generated by using the provided code - ExtractMetaData.py",
                "inputMode": "REQUIRED",
                "autoMountLocal": True,
                "sourceUrl": "https://drive.google.com/file/d/1HOgA3l_JRGYTGvpCdZ4nqYBE_KTS_kL1/view?usp=sharing",
                "targetPath": "dnnmodal.json"
                },
                {
                "name": "DNN_training_metadata",
                "description": "The JSON File generated by using the provided code - ExtractMetaData.py",
                "inputMode": "REQUIRED",
                "autoMountLocal": True,
                "sourceUrl": "https://drive.google.com/file/d/1HOgA3l_JRGYTGvpCdZ4nqYBE_KTS_kL1/view?usp=drive_link"
                "targetPath": "trainindata.json"
                },
                 {
                "name": "Systems_metadata",
                "description": "The JSON File generated with list of all systems",
                "inputMode": "REQUIRED",
                "autoMountLocal": True,
                "sourceUrl": "https://drive.google.com/file/d/1HOgA3l_JRGYTGvpCdZ4nqYBE_KTS_kL1/view?usp=drive_link"
                "targetPath": "systemsConfig.json"
                }
            ],
            # or put in Notes
        }
        # Submit a job
        job_response_hpc=self.TAPISclient.jobs.submitJob(name='Predict_DNN_Requirments',
                                                         description='Regression Esytimation',
                                                         appId=self.app_def.id,
                                                         execSystemId=systemID,
                                                         appVersion= '1.0.0',
                                                         **jobargs)

                                                        # Ecex system opt -> srchine dir or syste,

        return job_response_hpc

    def submitResoirceEstimator(self, model, systems):

        # Functions to push the systems.json Model.json and Training metadata into a webaccesible drive
        job_response_hpc = self.createJobForEstimation()

        # ask the user to look into a file
        job_status = self.checkJobStatus(job_response_hpc)

        if job_status == "COMPLETE":
            csv_res = self.TAPISclient.jobs.getJobOutputDownload(jobUuid=job_response_hpc,
                                                                                  outputPath='tapisjob.out')
            string_res = str(csv_res)
            # will have to have templated write and parse it here
            string_res = re.sub(r'\\n', '\n', string_res)
            return "Read the or display the CSV - string_res"
        
        else:
            return "waiting for estimation, try calling <another function"




    def checkJobStatus(self, jobID):
        Job_status = self.TAPISclient.jobs.getJobStatus(jobUuid=jobID)
        return Job_status


    def getResouceEstimation(self, model):
        executeOptions = self.getSystems()
        self.submitResoirceEstimator(model, executeOptions)

        return executeOptions
       
 


In [27]:
dnnhelper = DNNHelper(signIn = True, credsFile="TAPISCreds.json")

In [28]:
dnnhelper.getResouceEstimation(None)

-=-=-= [{'id': 'pitzer-OSC-harp-swathivm_1', 'host': 'pitzer.osc.edu', 'owner': 'swathivm', 'effectiveUserId': 'swathivm', 'systemType': 'LINUX', 'CPUType': 'abc', 'GPUType': 'xyz'}, {'id': 'OSC-pitzer-all-queues-harp-swathivm_case1', 'host': 'pitzer.osc.edu', 'owner': 'swathivm', 'effectiveUserId': 'swathivm', 'systemType': 'LINUX', 'CPUType': 'abc', 'GPUType': 'xyz'}, {'id': 'TACC-stampede2-all-queues-harp-swathivm_TEST1', 'host': 'stampede2.tacc.utexas.edu', 'owner': 'swathivm', 'effectiveUserId': 'swathivm', 'systemType': 'LINUX', 'CPUType': 'abc', 'GPUType': 'xyz'}, {'id': 'TACC-stampede2-all-queues-harp-swathivm_TEST101', 'host': 'stampede2.tacc.utexas.edu', 'owner': 'swathivm', 'effectiveUserId': 'swathivm', 'systemType': 'LINUX', 'CPUType': 'abc', 'GPUType': 'xyz'}, {'id': 'OSC-pitzer-all-queues-harp-swathivm_TEST2', 'host': 'pitzer.osc.edu', 'owner': 'swathivm', 'effectiveUserId': 'swathivm', 'systemType': 'LINUX', 'CPUType': 'abc', 'GPUType': 'xyz'}, {'id': 'TACC-stampede2-al

[{'id': 'pitzer-OSC-harp-swathivm_1',
  'host': 'pitzer.osc.edu',
  'owner': 'swathivm',
  'effectiveUserId': 'swathivm',
  'systemType': 'LINUX',
  'CPUType': 'abc',
  'GPUType': 'xyz'},
 {'id': 'OSC-pitzer-all-queues-harp-swathivm_case1',
  'host': 'pitzer.osc.edu',
  'owner': 'swathivm',
  'effectiveUserId': 'swathivm',
  'systemType': 'LINUX',
  'CPUType': 'abc',
  'GPUType': 'xyz'},
 {'id': 'TACC-stampede2-all-queues-harp-swathivm_TEST1',
  'host': 'stampede2.tacc.utexas.edu',
  'owner': 'swathivm',
  'effectiveUserId': 'swathivm',
  'systemType': 'LINUX',
  'CPUType': 'abc',
  'GPUType': 'xyz'},
 {'id': 'TACC-stampede2-all-queues-harp-swathivm_TEST101',
  'host': 'stampede2.tacc.utexas.edu',
  'owner': 'swathivm',
  'effectiveUserId': 'swathivm',
  'systemType': 'LINUX',
  'CPUType': 'abc',
  'GPUType': 'xyz'},
 {'id': 'OSC-pitzer-all-queues-harp-swathivm_TEST2',
  'host': 'pitzer.osc.edu',
  'owner': 'swathivm',
  'effectiveUserId': 'swathivm',
  'systemType': 'LINUX',
  'CPUTyp